In [32]:
import os
import importlib  # used to reload modules if necessary
path_to_frontend = '..'
os.chdir(path_to_frontend)
import q100viz.session as session
import q100viz.devtools as devtools
import q100viz.graphics.graphs as graphs

In [33]:
# prepare simulation:
devtools.select_buildings_for_simulation(session.buildings.df, ['7.31', '7.32', '7.53', '7.54'])
# session.buildings.df['selected'] = True

id
7.32    1.0
7.31    1.0
7.54    0.0
7.53    2.0
Name: group, dtype: float64


In [36]:
# start simulation:
session.simulation.activate(max_year=2045)

'/home/dunland/github/qScope'

In [ ]:
# create combined emissions graph:
importlib.reload(graphs)
graphs.export_combined_emissions(session.buildings.list_from_groups(), current_output_folder="/home/dunland/github/qScope/data/outputs/output_test", compare_data_folder="/home/dunland/github/qScope/data/outputs/output_bestand")

In [ ]:
# create combined energy prices graph:
importlib.reload(graphs)
graphs.export_combined_energy_prices("/home/dunland/github/qScope/data/outputs/output_test", outfile=None, compare_data_folder="/home/dunland/github/qScope/data/outputs/output_bestand")

In [ ]:
# create individual graphs:
importlib.reload(graphs)
output_folder = "/home/dunland/github/qScope/data/outputs/output_test"
for group_df in session.buildings.list_from_groups():
    if group_df is not None:
        for idx in group_df.index:

            # export emissions graph:
            graphs.export_using_columns(
                csv_name="/emissions/CO2_emissions_{0}.csv".format(
                    idx),
                data_folders=[output_folder],
                columns=['building_emissions'],
                title_="CO2-Emissionen",
                outfile=None,
                xlabel_="Jahr",
                ylabel_="ø-Emissionen [$kg_{CO2,eq}$]",
                x_='current_date',
                convert_grams_to_kg=True,
                compare_data_folder=output_folder + "/../output_bestand"
            )

In [ ]:
# investigate buildings dataframe:
df = session.buildings.df[['address', 'spec_heat_consumption', 'avg_spec_heat_consumption', 'type']]
devtools.print_full_df(df)
df.to_csv('buildings_list_export_2022-10_11_16-25.csv')
print(abs(session.buildings.df['spec_heat_consumption'] - session.buildings.df['avg_spec_heat_consumption']).describe())

In [ ]:
session.buildings.df[['address', 'type', 'cluster_size', 'avg_spec_heat_consumption', 'avg_spec_power_consumption']].to_csv("buildings_average_consumption_2022-10-13_09-18.csv")

In [ ]:
# make clusters and get average cluster data:
buildings_cluster = session.buildings.make_clusters(start_interval=0.5)
for j in range(len(session.buildings.df)):
    session.buildings.df.at[session.buildings.df.index[j], 'avg_spec_heat_consumption'] = buildings_cluster[j]['spec_heat_consumption'].mean()
    session.buildings.df.at[session.buildings.df.index[j], 'avg_spec_power_consumption'] = buildings_cluster[j]['spec_power_consumption'].mean()
    session.buildings.df.at[session.buildings.df.index[j], 'cluster_size'] = int(len(buildings_cluster[j]))
session.buildings.df[['cluster_size', 'spec_heat_consumption', 'avg_spec_heat_consumption', 'spec_power_consumption', 'avg_spec_power_consumption']].describe()